In [203]:
import os
import pandas as pd
import bokeh.plotting
import bokeh.io
import bebi103
from constants import H2, ALA, VAL, LEU, ILE, PRO, ASP, PHE, GLU, MET, SER, LYS, TYR
bokeh.io.output_notebook()


peaks_liklihood = {H2: 4, ALA: 1, VAL: 1, LEU: 1, ILE: 1, PRO:1, ASP:1, PHE:1,  GLU: 0.9,  MET:0.9,  SER: .75, LYS: 0.67, TYR: 0.67}



data_path = '../data/shay-2022-12-02'
filename = 'EN_HBG_data_batch2.xlsx'
original_df = pd.read_excel(os.path.join(data_path, filename), 'd2H_data')
df = original_df.rename(columns={"Identifier 1": "sample_id", "Rt": "relative_time", "Area 2": "area_2", "Area 3": "area_3", "Ampl  2": "amplitude_2", "Ampl  3": "amplitude_3", 'd 3H2/2H2':"d_3H2/2H2", 'R 3H2/2H2': "3H2_2H2_ratio" , 'd 2H/1H':"d_2H/1H"})

df.drop(["Rt.1"], axis=1, inplace=True)
def create_bio_replicate_col(sample_id):
    if sample_id == 'F8':
        return CONTROL
    else:
        return sample_id[:-1]
df['bio_replicate'] = df['sample_id'].apply(create_bio_replicate_col)
df['TENTATIVE_COMPOUND'] = None
def create_short_name_col(row):
    if row['bio_replicate'] == CONTROL:
        return 100
    else:
        return int(row['bio_replicate'][-2:])

def create_original_peak_cols(grp):
    grp['original_peak_order'] = range(0, len(grp))
    grp['original_peak_count'] = len(grp)
    return grp

def reset_peak_order_col(grp):
    grp['peak_order'] = range(0, len(grp))
    grp['peak_count'] = len(grp)
    return grp

def update_peak_order(df):
    print("Updating Peak Order", df.shape)
    df.drop(df.index[df['TENTATIVE_COMPOUND'] == NAPOI], inplace=True)
    print("after", df.shape)
    df = df.groupby('Analysis', sort='relative_tine', group_keys=False).apply(reset_peak_order_col)
    return df



def update_color_column(df, column_name='Analysis'):
    df['color'] = bebi103.viz.q_to_color(df[column_name], colors=bokeh.palettes.Category20_10)
    return df


df = df.groupby('Analysis', sort='relative_tine', group_keys=False).apply(create_original_peak_cols)
df['bio_replicate_id'] = df.apply(create_short_name_col, axis=1)

df.drop(df.index[df['bio_replicate'] == CONTROL], inplace=True)

df = update_color_column(df, "Analysis")
df = handle_first_2_h2(df)
df = handle_last_2_h2(df)
df = handle_alanine(df)
def create_time_relative_to_alanine_col(grp):
    alanine_time = int(grp[grp['TENTATIVE_COMPOUND'] == ALA]['relative_time'])
    grp['TIME_RELATIVE_TO_ALANINE'] = grp['relative_time'] -alanine_time
    return grp

df = df.groupby('Analysis', group_keys=False).apply(create_time_relative_to_alanine_col)
df

Loading BokehJS ...

/Users/madisond/Desktop/lab_work/orphan/isotope_ratio_mass_spec/env/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Updating Peak Order (2455, 20)
after (2327, 20)
Updating Peak Order (2327, 22)
after (2327, 22)
Updating Peak Order (2327, 22)
after (2327, 22)


,Date,Time,sample_id,relative_time,area_2,area_3,amplitude_2,amplitude_3,d_3H2/2H2,3H2_2H2_ratio,...,bio_replicate,TENTATIVE_COMPOUND,original_peak_order,original_peak_count,bio_replicate_id,color,index_col,peak_order,peak_count,TIME_RELATIVE_TO_ALANINE
0,07/24/22,16:37:07,EN-HBG-21a,705.0,59.543,0.019,3076,1033,-225.341,0.000241,...,EN-HBG-21,H2,0,24,21,#1f77b4,0,0,23,-327.0
1,07/24/22,16:37:07,EN-HBG-21a,817.2,60.205,0.019,3130,1056,-224.000,0.000242,...,EN-HBG-21,H2,2,24,21,#1f77b4,2,1,23,-214.8
2,07/24/22,16:37:07,EN-HBG-21a,1032.7,11.965,0.004,1496,473,-105.694,0.000279,...,EN-HBG-21,Alanine,3,24,21,#1f77b4,3,2,23,0.7
3,07/24/22,16:37:07,EN-HBG-21a,1304.0,13.214,0.005,1946,588,-127.019,0.000272,...,EN-HBG-21,None,4,24,21,#1f77b4,4,3,23,272.0
4,07/24/22,16:37:07,EN-HBG-21a,1468.0,14.339,0.005,1912,608,-122.756,0.000273,...,EN-HBG-21,None,5,24,21,#1f77b4,5,4,23,436.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322,07/28/22,21:19:17,EN-HBG-34d,2845.7,3.059,0.001,583,173,-112.035,0.000277,...,EN-HBG-34,None,35,40,34,#c5b0d5,2612,33,38,1827.7
2323,07/28/22,21:19:17,EN-HBG-34d,2872.7,0.454,0.000,68,24,-291.393,0.000221,...,EN-HBG-34,None,36,40,34,#c5b0d5,2613,34,38,1854.7
2324,07/28/22,21:19:17,EN-HBG-34d,3281.9,0.388,0.000,75,25,-126.250,0.000272,...,EN-HBG-34,None,37,40,34,#c5b0d5,2614,35,38,2263.9
2325,07/28/22,21:19:17,EN-HBG-34d,3433.0,65.077,0.018,3437,1064,-223.541,0.000242,...,EN-HBG-34,H2,38,40,34,#c5b0d5,2615,36,38,2415.0


In [130]:
first_six = df[(df['bio_replicate_id'] != CONTROL) & (df['original_peak_order'] <= 5)] 
p = bokeh.plotting.figure(
    width=800,
    height=300,
    x_axis_label="Peak Order",
    y_axis_label="Amplitude",
    toolbar_location='above',
)
p.circle(
    source=first_six,
    x="original_peak_order",
    y="amplitude_2",
    color='color'
)
bokeh.io.show(p)

In [116]:
# three_peaks_post_h2 = df[(df['peak_order'] >= 2) & (df['peak_order'] <= 4)]
one_peak = df[df['peak_order']==2]
# for grp, data in one_peak.groupby('bio_replicate_id'):
#     print("bio_group=", grp)
#     print(data[['relative_time', 'amplitude_2', 'peak_order']])
one_peak['amplitude_2'].min()

949

In [130]:
def set_valines(df):
    source_of_truth = df[df['Analysis']==4167][['TIME_RELATIVE_TO_ALANINE', 'peak_order', 'compound', 'amplitude_2', 'area_2', 'bio_replicate_id', 'TENTATIVE_COMPOUND', 'color']]
    valine_time = int(source_of_truth[source_of_truth['compound'] == 'Val']['TIME_RELATIVE_TO_ALANINE'].values[0])

    possible_valines = df[(df['TIME_RELATIVE_TO_ALANINE'] <= valine_time + 6) & (df['TIME_RELATIVE_TO_ALANINE'] >= valine_time - 6)]
    df.loc[possible_valines.index, ['TENTATIVE_COMPOUND']] = VAL
    return df

def set_leucines(df):
    source_of_truth = df[df['Analysis']==4167][['TIME_RELATIVE_TO_ALANINE', 'peak_order', 'compound', 'amplitude_2', 'area_2', 'bio_replicate_id', 'TENTATIVE_COMPOUND', 'color']]
    leucine_time = int(source_of_truth[source_of_truth['compound'] == 'Leu']['TIME_RELATIVE_TO_ALANINE'].values[0])
    possible_leucine = df[(df['TIME_RELATIVE_TO_ALANINE'] <= leucine_time + 2) & (df['TIME_RELATIVE_TO_ALANINE'] >= leucine_time - 3)]
    df.loc[possible_leucine.index, ['TENTATIVE_COMPOUND']] = LEU
    return df

def set_isoleucines(df):
    source_of_truth = df[df['Analysis']==4167][['TIME_RELATIVE_TO_ALANINE', 'peak_order', 'compound', 'amplitude_2', 'area_2', 'bio_replicate_id', 'TENTATIVE_COMPOUND', 'color']]
    isoleucine_time = int(source_of_truth[source_of_truth['compound'] == 'Ile']['TIME_RELATIVE_TO_ALANINE'].values[0])
    possible_isoleucine = df[(df['TIME_RELATIVE_TO_ALANINE'] <= isoleucine_time + 5) & (df['TIME_RELATIVE_TO_ALANINE'] >= isoleucine_time - 5) & (df['amplitude_2'] >= 600)]
    df.loc[possible_isoleucine.index, ['TENTATIVE_COMPOUND']] = ILE
    # this is basically the same functionality as the two lines above -- just taking the max instead of setting a cut off
    # possible_isoleucine = df[(df['TIME_RELATIVE_TO_ALANINE'] <= isoleucine_time + 5) & (df['TIME_RELATIVE_TO_ALANINE'] >= isoleucine_time - 5)]
    # isoleucine_index_ids = []
    # for group, data in possible_isoleucine.groupby('Analysis'):
    #     if len(data) ==1:
    #         isoleucine_index_ids.append(data.index[0])
            
    #     else:
    #         isoleucine_index_ids.append(data['amplitude_2'].idxmax())    
    # df.loc[isoleucine_index_ids, ['TENTATIVE_COMPOUND']] = ILE
    return df

def set_prolines(df):
    source_of_truth = df[df['Analysis']==4167][['TIME_RELATIVE_TO_ALANINE', 'peak_order', 'compound', 'amplitude_2', 'area_2', 'bio_replicate_id', 'TENTATIVE_COMPOUND', 'color']]
    proline_time = int(source_of_truth[source_of_truth['compound'] == 'Pro']['TIME_RELATIVE_TO_ALANINE'].values[0])
    possible_proline = df[(df['TIME_RELATIVE_TO_ALANINE'] <= proline_time + 5) & (df['TIME_RELATIVE_TO_ALANINE'] >= proline_time - 5)]
    df.loc[possible_proline.index, ['TENTATIVE_COMPOUND']] = PRO
    return df

def set_aspartic_acid(df):
    source_of_truth = df[df['Analysis']==4167][['TIME_RELATIVE_TO_ALANINE', 'peak_order', 'compound', 'amplitude_2', 'area_2', 'bio_replicate_id', 'TENTATIVE_COMPOUND', 'color']]
    aspartic_acid_time = int(source_of_truth[source_of_truth['compound'] == 'Asp']['TIME_RELATIVE_TO_ALANINE'].values[0])
    possible_asp_acid = df[(df['TIME_RELATIVE_TO_ALANINE'] <= aspartic_acid_time + 9) & (df['TIME_RELATIVE_TO_ALANINE'] >= aspartic_acid_time -3) & (df['amplitude_2'] >= 700)]
    df.loc[possible_asp_acid.index, ['TENTATIVE_COMPOUND']] = ASP
    return df

def set_phenylalanine(df):
    source_of_truth = df[df['Analysis']==4167][['TIME_RELATIVE_TO_ALANINE', 'peak_order', 'compound', 'amplitude_2', 'area_2', 'bio_replicate_id', 'TENTATIVE_COMPOUND', 'color']]
    phenylalanine_time = int(source_of_truth[source_of_truth['compound'] == 'Phe']['TIME_RELATIVE_TO_ALANINE'].values[0])
    possible_phenylalanine = df[(df['TIME_RELATIVE_TO_ALANINE'] <= phenylalanine_time + 4) & (df['TIME_RELATIVE_TO_ALANINE'] >= phenylalanine_time - 6) & (df['amplitude_2'] >= 500)]
    df.loc[possible_phenylalanine.index, ['TENTATIVE_COMPOUND']] = PHE
    return df

def set_glutamic_acid(df):
    source_of_truth = df[df['Analysis']==4167][['TIME_RELATIVE_TO_ALANINE', 'peak_order', 'compound', 'amplitude_2', 'area_2', 'bio_replicate_id', 'TENTATIVE_COMPOUND', 'color']]
    glutamic_acid_time = int(source_of_truth[source_of_truth['compound'] == 'Glu']['TIME_RELATIVE_TO_ALANINE'].values[0])
    possible_glutamic_acid = df[(df['TIME_RELATIVE_TO_ALANINE'] <= glutamic_acid_time + 2) & (df['TIME_RELATIVE_TO_ALANINE'] >= glutamic_acid_time -6)]
    df.loc[possible_glutamic_acid.index, ['TENTATIVE_COMPOUND']] = GLU
    return df


def label_aas(df):
    df = set_valines(df)
    df = set_leucines(df)
    df = set_isoleucines(df)
    df = set_prolines(df)
    df = set_aspartic_acid(df)
    df = set_phenylalanine(df)
    df = set_glutamic_acid(df)
    return df
df = label_aas(df)
df.head(10)

,Date,Time,sample_id,relative_time,area_2,area_3,amplitude_2,amplitude_3,d_3H2/2H2,3H2_2H2_ratio,...,bio_replicate,TENTATIVE_COMPOUND,original_peak_order,original_peak_count,bio_replicate_id,color,index_col,peak_order,peak_count,TIME_RELATIVE_TO_ALANINE
0,07/24/22,16:37:07,EN-HBG-21a,705.0,59.543,0.019,3076,1033,-225.341,0.000241,...,EN-HBG-21,H2,0,24,21,#1f77b4,0,0,23,-327.0
1,07/24/22,16:37:07,EN-HBG-21a,817.2,60.205,0.019,3130,1056,-224.000,0.000242,...,EN-HBG-21,H2,2,24,21,#1f77b4,2,1,23,-214.8
2,07/24/22,16:37:07,EN-HBG-21a,1032.7,11.965,0.004,1496,473,-105.694,0.000279,...,EN-HBG-21,Alanine,3,24,21,#1f77b4,3,2,23,0.7
3,07/24/22,16:37:07,EN-HBG-21a,1304.0,13.214,0.005,1946,588,-127.019,0.000272,...,EN-HBG-21,Valine,4,24,21,#1f77b4,4,3,23,272.0
4,07/24/22,16:37:07,EN-HBG-21a,1468.0,14.339,0.005,1912,608,-122.756,0.000273,...,EN-HBG-21,Leucine,5,24,21,#1f77b4,5,4,23,436.0
5,07/24/22,16:37:07,EN-HBG-21a,1493.9,9.503,0.003,1693,476,-203.901,0.000248,...,EN-HBG-21,Isoleucine,6,24,21,#1f77b4,6,5,23,461.9
6,07/24/22,16:37:07,EN-HBG-21a,1523.8,5.293,0.002,167,62,-147.770,0.000266,...,EN-HBG-21,None,7,24,21,#1f77b4,7,6,23,491.8
7,07/24/22,16:37:07,EN-HBG-21a,1567.3,10.879,0.004,1490,542,-30.260,0.000302,...,EN-HBG-21,Proline,8,24,21,#1f77b4,8,7,23,535.3
8,07/24/22,16:37:07,EN-HBG-21a,1706.5,11.410,0.004,1336,420,-129.420,0.000271,...,EN-HBG-21,None,9,24,21,#1f77b4,9,8,23,674.5
9,07/24/22,16:37:07,EN-HBG-21a,1713.6,0.761,0.000,113,30,-385.780,0.000191,...,EN-HBG-21,None,10,24,21,#1f77b4,10,9,23,681.6


In [197]:

# Glutamic acid"
# MET:0.9,  SER: .75, LYS: 0.67, TYR: 0.67}

source_of_truth = df[df['Analysis']==4167][['TIME_RELATIVE_TO_ALANINE', 'peak_order', 'compound', 'amplitude_2', 'area_2', 'bio_replicate_id', 'TENTATIVE_COMPOUND', 'color']]

glutamic_acid_time = int(source_of_truth[source_of_truth['compound'] == 'Glu']['TIME_RELATIVE_TO_ALANINE'].values[0])
possible_glutamic_acid = df[(df['TIME_RELATIVE_TO_ALANINE'] <= glutamic_acid_time + 2) & (df['TIME_RELATIVE_TO_ALANINE'] >= glutamic_acid_time -6)]
df.loc[possible_glutamic_acid.index, ['TENTATIVE_COMPOUND']] = GLU
print(len(possible_glutamic_acid['sample_id'].values))
print(len(possible_glutamic_acid['sample_id'].unique()))
print(len(df['sample_id'].unique()))
# possible_glutamic_acid[['amplitude_2', 'area_2']].describe()
# source_of_truth[source_of_truth['compound'] == 'Glu']




76
76
76


,amplitude_2,area_2
count,76.000000,76.000000
mean,830.197368,5.208368
std,186.374892,1.585184
min,395.000000,1.931000
25%,735.500000,4.190000
50%,846.500000,5.422500
75%,946.500000,6.315500
max,1404.000000,11.133000


,Date,Time,sample_id,relative_time,area_2,area_3,amplitude_2,amplitude_3,d_3H2/2H2,3H2_2H2_ratio,...,bio_replicate,TENTATIVE_COMPOUND,original_peak_order,original_peak_count,bio_replicate_id,color,index_col,peak_order,peak_count,TIME_RELATIVE_TO_ALANINE
0,07/24/22,16:37:07,EN-HBG-21a,705.0,59.543,0.019,3076,1033,-225.341,0.000241,...,EN-HBG-21,H2,0,24,21,#1f77b4,0,0,23,-327.0
1,07/24/22,16:37:07,EN-HBG-21a,817.2,60.205,0.019,3130,1056,-224.000,0.000242,...,EN-HBG-21,H2,2,24,21,#1f77b4,2,1,23,-214.8
2,07/24/22,16:37:07,EN-HBG-21a,1032.7,11.965,0.004,1496,473,-105.694,0.000279,...,EN-HBG-21,Alanine,3,24,21,#1f77b4,3,2,23,0.7
3,07/24/22,16:37:07,EN-HBG-21a,1304.0,13.214,0.005,1946,588,-127.019,0.000272,...,EN-HBG-21,Valine,4,24,21,#1f77b4,4,3,23,272.0
4,07/24/22,16:37:07,EN-HBG-21a,1468.0,14.339,0.005,1912,608,-122.756,0.000273,...,EN-HBG-21,None,5,24,21,#1f77b4,5,4,23,436.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322,07/28/22,21:19:17,EN-HBG-34d,2845.7,3.059,0.001,583,173,-112.035,0.000277,...,EN-HBG-34,None,35,40,34,#c5b0d5,2612,33,38,1827.7
2323,07/28/22,21:19:17,EN-HBG-34d,2872.7,0.454,0.000,68,24,-291.393,0.000221,...,EN-HBG-34,None,36,40,34,#c5b0d5,2613,34,38,1854.7
2324,07/28/22,21:19:17,EN-HBG-34d,3281.9,0.388,0.000,75,25,-126.250,0.000272,...,EN-HBG-34,None,37,40,34,#c5b0d5,2614,35,38,2263.9
2325,07/28/22,21:19:17,EN-HBG-34d,3433.0,65.077,0.018,3437,1064,-223.541,0.000242,...,EN-HBG-34,H2,38,40,34,#c5b0d5,2615,36,38,2415.0


In [220]:

from bokeh.models import ColumnDataSource, HoverTool

source_of_truth = df[df['Analysis']==4167][['TIME_RELATIVE_TO_ALANINE', 'peak_order', 'compound', 'amplitude_2', 'area_2', 'bio_replicate_id', 'color']]
p = bokeh.plotting.figure(
        width=800,
        height=300,
        x_axis_label="TIME_RELATIVE_TO_ALANINE",
        y_axis_label="Amplitude",
        toolbar_location='above',
        # tools=[HoverTool(tooltips=[('Compound', '@compound')])]
        )
p.circle(
    source=df,
    x="TIME_RELATIVE_TO_ALANINE",
    y="area_2",
    color='color',
)
p.circle(
    source=source_of_truth,
    x="TIME_RELATIVE_TO_ALANINE",
    y="area_2",
    size=40,
    color='red',
    alpha=0.1

)

bokeh.io.show(p)

In [35]:
df['TENTATIVE_COMPOUND'].unique()

array([None], dtype=object)

In [19]:
titles = [24, 25, 30]

fig_list = []
for x in titles:
    subset = df[df['bio_replicate_id'] == x]
    fig_list.append(plot_time_relative_to_alanine_V_amplitude_2(subset))


# Generating tab image from list
tabs = [Panel(child=fig_list[l],title=str(titles[l]) ) for l in range(len(fig_list))]
bokeh.io.show(Tabs(tabs=tabs))

In [5]:
## label based on alanine rt
## label based on time relative to last labeled aa

df

,Date,Time,sample_id,relative_time,area_2,area_3,amplitude_2,amplitude_3,d_3H2/2H2,3H2_2H2_ratio,...,compound,bio_replicate,TENTATIVE_COMPOUND,original_peak_order,original_peak_count,bio_replicate_id,color,index_col,peak_order,peak_count
0,07/24/22,16:37:07,EN-HBG-21a,705.0,59.543,0.019,3076,1033,-225.341,0.000241,...,H2,EN-HBG-21,H2,0,24,21,#1f77b4,0,0,23
1,07/24/22,16:37:07,EN-HBG-21a,817.2,60.205,0.019,3130,1056,-224.000,0.000242,...,H2,EN-HBG-21,H2,2,24,21,#1f77b4,2,1,23
2,07/24/22,16:37:07,EN-HBG-21a,1032.7,11.965,0.004,1496,473,-105.694,0.000279,...,Ala,EN-HBG-21,None,3,24,21,#1f77b4,3,2,23
3,07/24/22,16:37:07,EN-HBG-21a,1304.0,13.214,0.005,1946,588,-127.019,0.000272,...,Val,EN-HBG-21,None,4,24,21,#1f77b4,4,3,23
4,07/24/22,16:37:07,EN-HBG-21a,1468.0,14.339,0.005,1912,608,-122.756,0.000273,...,Leu,EN-HBG-21,None,5,24,21,#1f77b4,5,4,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322,07/28/22,21:19:17,EN-HBG-34d,2845.7,3.059,0.001,583,173,-112.035,0.000277,...,NaN,EN-HBG-34,None,35,40,34,#c5b0d5,2612,33,38
2323,07/28/22,21:19:17,EN-HBG-34d,2872.7,0.454,0.000,68,24,-291.393,0.000221,...,NaN,EN-HBG-34,None,36,40,34,#c5b0d5,2613,34,38
2324,07/28/22,21:19:17,EN-HBG-34d,3281.9,0.388,0.000,75,25,-126.250,0.000272,...,NaN,EN-HBG-34,None,37,40,34,#c5b0d5,2614,35,38
2325,07/28/22,21:19:17,EN-HBG-34d,3433.0,65.077,0.018,3437,1064,-223.541,0.000242,...,NaN,EN-HBG-34,H2,38,40,34,#c5b0d5,2615,36,38
